In [131]:
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
from helpers import *

save_dir = "downloaded_documents"

In [20]:
def save_documents_dm_apps():
    doc_urls = sql("SELECT library_link_en FROM csas2_document WHERE url_en IS NOT NULL")
    os.makedirs(save_dir, exist_ok=True)

    for url in doc_urls.drop_duplicates().library_link_en.to_list():
        try:
            response = requests.get(url)
            if response.status_code == 200:
                file_name = os.path.join(save_dir, url.split('/')[-1])
                with open(file_name, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded: {url}")
            else:
                print(f"Skipped (HTTP {response.status_code}): {url}")
        except Exception as e:
            print(f"Error with {url}: {e}")

# save_documents_dm_apps()

In [120]:
documents = pd.read_csv('prod-publication.txt', delimiter='|', encoding='latin-1')
document_list = documents['URL_ENGLISH'].drop_duplicates().to_list()
document_list = [x.replace('html', 'htm').replace('\t ', '').strip() for x in document_list if type(x) == str if x not in ['3', '4', '6', '7']]

In [137]:
def download(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(os.path.join(save_dir, url.split('/')[-1]), 'wb') as f:
                f.write(response.content)
            print(f"\tDownloaded: {url}")
        else:
            print(f"\tSkipped (HTTP {response.status_code}): {url}")
    except Exception as e:
        print(f"\tError with {url}: {e}")
        
        
def get_pdf_url_from_html_url(url):
    url_contains = 'waves-vagues.dfo-mpo.gc.ca'
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            for link in soup.find_all('a', href=True):
                href = link['href']
                if url_contains in href:
                    return href
        else:
            print(f"\tCould not access page (HTTP {response.status_code}): {url}")
    except Exception as e:
        print(f"\tError accessing {url}: {e}")
    
    return None

def save_documents_csas_website():
    for doc_url in document_list:
        print(f"Processing document: {doc_url}")
        pdf_url = None
        file_name = os.path.join(save_dir, doc_url.split('/')[-1])
        if file_name.split('.')[-1] == 'pdf':
            pdf_url = doc_url
        else:
            pdf_url = get_pdf_url_from_html_url(doc_url)
    
        if os.path.exists(file_name):
            print(f"\tFile already exists, skipping: {file_name}")
        elif pdf_url:
            download(pdf_url)
            
# save_documents_csas_website()

Processing document: http://waves-vagues.dfo-mpo.gc.ca/Library/286840.pdf
	File already exists, skipping: downloaded_documents\286840.pdf
Processing document: http://waves-vagues.dfo-mpo.gc.ca/Library/315362.pdf
	File already exists, skipping: downloaded_documents\315362.pdf
Processing document: http://www.dfo-mpo.gc.ca/csas-sccs/Publications/Pro-Cr/2004/2004_046-eng.htm
	Downloaded: https://waves-vagues.dfo-mpo.gc.ca/library-bibliotheque/317180.pdf
Processing document: http://waves-vagues.dfo-mpo.gc.ca/Library/315357.pdf
	File already exists, skipping: downloaded_documents\315357.pdf
Processing document: http://waves-vagues.dfo-mpo.gc.ca/Library/315359.pdf
	File already exists, skipping: downloaded_documents\315359.pdf
Processing document: http://waves-vagues.dfo-mpo.gc.ca/Library/315111.pdf
	File already exists, skipping: downloaded_documents\315111.pdf
Processing document: http://waves-vagues.dfo-mpo.gc.ca/Library/315147.pdf
	File already exists, skipping: downloaded_documents\31514